In [119]:
import pandas as pd
import joblib
from pathlib import Path
from typing import Literal

from data_processor import process_datasets

## Define recommender

In [146]:
class Recommender:
    data_idx_mapping = {
        "2080": 1, 
        "disjoint": 2
    }
    def __init__(self, 
                 estimator: Literal["2080", "disjoint"] = "disjoint",
                 models_dir: Path | str = Path("../models"),
                 data_dir: Path | str = Path("../data/interim"),
                 raw_data_path: Path | str = Path("../data/raw/ml-100k")
                 ):
        self.estimator = estimator
        self.models_dir = models_dir
        self.data_dir = data_dir
        self.raw_data_path = raw_data_path
        self.u_data_path = raw_data_path / "u.data"

        data_idx = self.data_idx_mapping[estimator]
        self.train_x = pd.read_parquet(data_dir / f"train_x{data_idx}.parquet")
        self.test_x = pd.read_parquet(data_dir / f"test_x{data_idx}.parquet")
        self.train_y = pd.read_parquet(data_dir / f"train_y{data_idx}.parquet")
        self.test_y = pd.read_parquet(data_dir / f"test_y{data_idx}.parquet")

        self.model = joblib.load(models_dir / f"model_{estimator}.pkl")

    def predict(self, data):
        return self.model.predict(data)
    
    def recommend(self, user_ids, number_of_recs: int = 5):
        item_df = pd.read_csv(self.raw_data_path / "u.item", sep= "|", encoding='latin-1', header=None)
        item_df = item_df.iloc[:,:2]
        item_df.columns = ["item_id", "title"]

        u_data = pd.read_csv(self.u_data_path, sep="\t", header=None)
        u_data.columns = ["user_id", "item_id", "rating", "timestamp"]
        
        u_data = u_data[u_data["user_id"].isin(user_ids)]
        
        recommend_x, recommend_y = process_datasets(u_data)
        ratings = self.predict(recommend_x)

        recommend_x[["user_id", "item_id"]] = u_data[["user_id", "item_id"]]
        recommendation = pd.concat([recommend_x.reset_index(), pd.Series(ratings, name="rating")], axis=1)
        recommendation.rating *= 5

        recommendation = recommendation.join(item_df, on="item_id", lsuffix="_drop")
        recommendation = recommendation[["user_id", "title", "rating", "item_id"]]
        recommendation = recommendation.groupby(by=["user_id"]).apply(lambda x: x.sort_values(by="rating", ascending=False)[:number_of_recs])

        return recommendation

In [147]:
recommender = Recommender()

These user ids weren't used for the training of disjoint model. We can pick any of them to get the movies recommendations using the model. The 2080 model is there but is not representative enough as some user_id-item_id combinations could be in the training set.
## Inference

In [148]:
user_ids = pd.read_parquet(recommender.data_dir / "test_user_ids.parquet")
recommendation = recommender.recommend(user_ids["user_id"][:5])
recommendation

  0%|          | 0/1681 [00:00<?, ?it/s]

  0%|          | 0/1681 [00:00<?, ?it/s]

  0%|          | 0/1680 [00:00<?, ?it/s]

  0%|          | 0/1680 [00:00<?, ?it/s]

C:\Users\Anthony\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\Anthony\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Users\Anthony\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_catego

user_id                                   title    rating  \
user_id                                                                  
160     259      160              Maltese Falcon, The (1941)  4.609278   
        412      160                             Jack (1996)  4.568492   
        102      160            It Happened One Night (1934)  4.540579   
        125      160                  Cinema Paradiso (1988)  4.516464   
        104      160              Legends of the Fall (1994)  4.497709   
264     213      264        Manchurian Candidate, The (1962)  4.757523   
        99       264                      Stand by Me (1986)  4.478101   
        80       264                 Forbidden Planet (1956)  4.394106   
        82       264                       Disclosure (1994)  4.257335   
        232      264                          Vertigo (1958)  4.214149   
459     403      459  Snow White and the Seven Dwarfs (1937)  5.391623   
        141      459             Hot Shots! Part Deux (1993)  5.222333   
        423      459                         Supercop (1992)  5.141069   
        133      459                       GoodFellas (1990)  5.111461   
        165      459                 Dead Man Walking (1995)  5.055168   
890     415      890              Legends of the Fall (1994)  4.437537   
        584      890                          Amadeus (1984)  4.313201   
        335      890                 Harold and Maude (1971)  4.243998   
        385      890                           Brazil (1985)  4.226839   
        337      890                 Army of Darkness (1993)  4.223931   
927     519      927      What's Eating Gilbert Grape (1993)  3.615865   
        359      927             Hot Shots! Part Deux (1993)  3.447083   
        505      927               Dead Poets Society (1989)  3.318901   
        378      927      Four Weddings and a Funeral (1994)  3.258021   
        558      927               Dances with Wolves (1990)  3.248543   

             item_id  
user_id               
160     259      484  
        412      409  
        102      604  
        125      170  
        104       51  
264     213      657  
        99       655  
        80       434  
        82        43  
        232      479  
459     403       99  
        141       80  
        423      128  
        133      182  
        165        9  
890     415       51  
        584      191  
        335      428  
        385      175  
        337      184  
927     519       65  
        359       80  
        505      196  
        378       70  
        558       97